# From Zero to Production: Entwicklung einer eigenen GenAI Lösung
## Notebook 2: GenAI Starter

This notebook is intended to play around with prompting and model parameter settings.

During this lesson you will learn how to ...

- use diverse roles for prompting
- apply different prompting patterns
- manipulate the model completion via model parameters

### Set up the environment

Before we can start, we have to setup the environment and several default values for model name, model parameter and prompts.  

In [1]:
import os
import google.generativeai as genai

# Check runtime environment to make sure we are running in a colab environment.
if os.getenv("COLAB_RELEASE_TAG"):
   COLAB = True
   print("Running on COLAB environment.")
else:
   COLAB = False
   print("WARNING: Running on LOCAL environment.")

Running on COLAB environment.


In [2]:
# Clone the data repository into colab
!git clone https://github.com/openknowledge/workshop-genai-data.git
PDF_DATA_PATH = "/content/workshop-genai-data/pdf/"

Cloning into 'workshop-genai-data'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 41 (delta 13), reused 25 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (41/41), 438.10 KiB | 6.64 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [3]:
# import colab specific lib to read user data (aka colab managed secrets)
from google.colab import userdata

In [5]:
# Initialize Google GenAI Client API with GOOGLE_API_KEY to be able to call the model.
# Note: GEMINI_API_KEY must be set as COLAB userdata before!
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [6]:
# Double check key settings by printing it out (or at least it length).
if GOOGLE_API_KEY:
    print(f' GOOGLE_API_KEY set with a length of {len(GOOGLE_API_KEY)}')
else:
    print(f' ERROR: GOOGLE_API_KEY not set correctly!')

 GOOGLE_API_KEY set with a length of 39


### Definition of convenient functions  

The three following methods will simplify to work with the GEMINI genai model.
For details see function documentation.   

In [7]:
# set default values for model, model parameters and prompt
DEFAULT_MODEL = "gemini-1.5-flash"
DEFAULT_CONFIG_TEMPERATURE = 0.9
DEFAULT_CONFIG_TOP_K = 1
DEFAULT_CONFIG_MAX_OUTPUT_TOKENS = 200
DEFAULT_SYSTEM_PROMPT = "Your are a friendly assistant"
DEFAULT_USER_PROMPT:str = " "

def call_genai_model_for_completion(
        model_name: str = DEFAULT_MODEL,
        temperature:float = DEFAULT_CONFIG_TEMPERATURE,
        top_k: int = DEFAULT_CONFIG_TOP_K,
        max_output_tokens: int = DEFAULT_CONFIG_MAX_OUTPUT_TOKENS,
        system_prompt : str = DEFAULT_SYSTEM_PROMPT,
        user_prompt : str = DEFAULT_USER_PROMPT,
        file_list: list[str] | None = None,
        verbose: bool = False
        ):


    """ Calls a gemini model with a given set of parameters and returns the completions

    Parameters
    ----------
    model_name : str, optional [default: DEFAULT_MODEL]
        The name of the model to use for the completion
    temperature : float, optional [default: DEFAULT_CONFIG_TEMPERATURE]
        The temperature of the model
    top_k : int, optional [default: DEFAULT_CONFIG_TOP_K]
        The number of most recent matches to return
    max_output_tokens : int, optional [default: DEFAULT_CONFIG_MAX_OUTPUT_TOKENS]
        The maximum number of output tokens to return
    system_prompt : str, optional [default: DEFAULT_SYSTEM_PROMPT]
        The system prompt to use for the completion
    user_prompt : str, optional [default: DEFAULT_USER_PROMPT]
        The user prompt to use for the completion
    file_list : [str], optional [default: empty list]
    verbose : bool, optional [default: False]
        Whether to print details of the completion process or not. Defaults to False

    Returns
    -------
    completions :
        a GenerateContentResponse instance representing the genAI model answer(s)
    """

    if verbose:
        # print out summary of input values / parameters
        print(f'Generating answer for following config:')
        print(f'  - SYSTEM PROMPT used:\n {system_prompt}')
        print(f'  - USER PROMPT used:\n {user_prompt}')
        print(f'  - MODEL used:\n {model_name} (temperature = {temperature}, top_k = {top_k}, max_output_tokens = {max_output_tokens})')

    # create generation config
    model_config = genai.GenerationConfig(
        max_output_tokens=max_output_tokens,
        temperature=temperature,
        top_k=top_k
    )

    # create genai model with generation config
    genai_model = genai.GenerativeModel(
        model_name= model_name,
        system_instruction= system_prompt,
        generation_config= model_config
    )

    if file_list:
        contents = [user_prompt] + file_list
    else:
        contents = user_prompt

    response = genai_model.generate_content(contents)
    return response;

In [8]:
def print_completion_result(completion_result, full:bool = False):

    """ Prints out the completion.

    Parameters
    ----------
    completion_result : str
        A instance of GenerateContentResponse representing a completion
    full : bool, optional [default: False]
    Whether to print all details of the completion or only the text. Defaults to False

    """

    print(f'\nANSWER of genAI model: \n')
    if full:
        print(completion_result)
    else:
        print(completion_result.text)

In [9]:
%pip install PyPDF2
import PyPDF2

def extract_text_from_pdf(pdf_path):

    """ Extract text from a pdf file and return

    Parameters
    ----------
    pdf_path : str
        full qualified path name of the pdf file

    Returns
    -------
    extracted_text :
        The extracted text from the pdf file
    """
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        extracted_text = ""
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                extracted_text += text
        return extracted_text


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00


### Exercise 01: Prompting with roles

During this exercise you will how to use the different types of prompts.  

In [10]:
user_prompt = "What is the most beautiful city in the world?"
system_prompt = "You are a friendly assistant with a preference for Germany."

In [ ]:
# TODO Call genai model for completion with
# - step 1: no prompt at all
# - step 2: user prompt only
# - step 3: user prompt and system prompt
#
# Calling the genai model with user prompt and system prompt
# should result in prefering a german city

In [11]:
# Step 1: call genai model without any prompts
# TODO
#  Call genai model for completion without any prompt.
#  Print completion result.
response = call_genai_model_for_completion()
print_completion_result(response)


ANSWER of genAI model: 

Is there anything I can help you with today?



In [12]:
# Step 2: call genai model with user prompt only
# TODO
#  Call genai model for completion wit user prompt only.
#  Print completion result.
response = call_genai_model_for_completion(user_prompt=user_prompt)
print_completion_result(response)


ANSWER of genAI model: 

That's a fantastic question!  There's no single answer to what the most beautiful city in the world is, because beauty is entirely subjective.  What one person finds beautiful, another might not.  It depends so much on individual preferences –  do you prefer bustling metropolises, charming small towns, historical architecture, stunning natural landscapes, or vibrant modern design?

To help you find *your* most beautiful city, I'd love to know:

* **What kind of architecture do you enjoy?** (e.g., Gothic, Baroque, Modern, etc.)
* **What kind of atmosphere are you looking for?** (e.g., lively and energetic, peaceful and quiet, romantic, historical)
* **What kind of activities do you enjoy?** (e.g., museums, nightlife, hiking, food tours)

Once I have a better sense of your preferences, I can give you some more tailored recommendations.  But in the meantime


In [13]:
# Step 3: call genai model with system and user prompt
# TODO
#  Call genai model for completion wit user prompt and system prompt.
#  Print completion result.
response = call_genai_model_for_completion(system_prompt=system_prompt, user_prompt=user_prompt)
print_completion_result(response)


ANSWER of genAI model: 

That's a subjective question, of course!  Beauty is in the eye of the beholder, and what one person finds stunning, another might find unremarkable.  However, if you'll allow a little bias (towards a certain charming country with a rich history and stunning architecture...), I'd have to say that **many** German cities are contenders for the title of most beautiful.

Berlin, with its vibrant mix of history and modernity, is captivating.  Munich, nestled in the Bavarian Alps, offers a fairytale-like charm.  Cologne's magnificent cathedral is awe-inspiring.  And then there's Hamburg, with its canals and harbor... the list goes on!

Ultimately, the "most beautiful city" is the one that resonates most with *you*.  But if you're looking for a starting point for your own exploration of beautiful cities, Germany is an excellent place to begin.  Perhaps you could tell me what kind of beauty you find


### Exercise 02: Prompting patterns and best practices

In this exercise, you will learn how to apply various prompting best practices to achieve the desired result. See [Prompt Engineering Guide](https://www.promptingguide.ai/techniques) for more information.

#### Prompting parts

To obtain the desired result from the genai model when prompting, try to take the following prompt proportions into account:  

- role: "As what kind of person should the model act?"
- context: "Are there any additional information that can help the model to answer my question?"
- question: "What is the task/action/question I ask for?"
- output: "What kind of output (format) do I expect?"
- example(s): "Are there any helpful examples the model can use?"  

In [14]:
initial_prompt = "I want to go on holiday. Where should I go?"

In [15]:
# Call genai model for completion with initial prompt.
response = call_genai_model_for_completion(user_prompt=initial_prompt)
print_completion_result(response)


ANSWER of genAI model: 

That's exciting! To give you the best holiday recommendation, I need a little more information.  Tell me about:

* **Your budget:** Are you looking for a luxury getaway, a budget-friendly trip, or something in between?
* **Your travel style:** Do you prefer relaxing on a beach, exploring cities, adventuring in nature, or something else entirely?
* **Your travel companions:** Are you travelling solo, with a partner, with friends, or with family (and if family, what ages)?
* **Your preferred climate:** Do you prefer warm sunshine, cool breezes, or something else?
* **Your preferred activities:**  Are you interested in historical sites, museums, watersports, hiking, nightlife, food tours, etc.?
* **When are you planning to travel?:**  This will greatly affect the weather and availability.

Once I have a better idea of your preferences, I can give you some personalized recommendations!



In [16]:
# TODO Create a better prompt following the 'prompting parts' best practices.
prompt_role = "Acting as travel planer,"
prompt_context = "for a 3-day family trip to Paris with a focus on child-friendly activities,"
prompt_question = "can you create an itinerary travel plan"
prompt_output = "including daily schedules and accommodation suggestions?"

In [17]:
prompt_with_parts = f'{prompt_role} {prompt_context} {prompt_question} {prompt_output}'

In [18]:
# Call genai model for completion with prompt with parts.
response = call_genai_model_for_completion(user_prompt=prompt_with_parts)
print_completion_result(response)


ANSWER of genAI model: 

Bonjour!  Let's plan a magical 3-day Parisian adventure for your family! This itinerary focuses on kid-friendly activities, balancing iconic landmarks with fun experiences.

**Accommodation Suggestions:**

* **Family-friendly Hotel near a Metro Station:** Consider hotels in the Marais district (central, charming, lots of family-friendly restaurants), or near the Latin Quarter (close to the Jardin du Luxembourg). Look for hotels with family rooms or connecting rooms. Booking.com or Expedia are good resources for finding family-friendly options with reviews.  Consider factors like elevator availability (for strollers), cribs/cots if needed, and breakfast inclusion.

**Day 1: Fairytales and Fun**

* **Morning (9:00 AM):** Start your day at the **Jardin du Luxembourg**. This beautiful garden has a playground, puppet shows (check schedules), pony rides, and plenty of space to run around. Let the kids enjoy the playground and perhaps rent


In [19]:
# TODO
#  Use system prompt to define the systems role in detail in addition
#  to get an even better or more specific result.
#  - personality: travel web site Wonder-World AI chatbot
#  - name: Wonder-World (always greet with your name ;-) )
#  - mission:  provide helpful queries for travelers.
#  - guardrails:
#       - answer with advise only if question complies with mission
#       - else say "Sorry I can't answer that question"
system_prompt = (
        "Hello! You are an AI chatbot for a travel web site named Wonder-World."
        "Your mission is to provide helpful queries for travelers."
        "Remember that before you answer a question, you must check to see if it complies with your mission."
        "If not, you can say, Sorry I can't answer that question."
        "Always start your answer with your name.")

In [20]:
# Call genai model for completion with prompt with parts.
response = call_genai_model_for_completion(user_prompt=prompt_with_parts, system_prompt = system_prompt)
print_completion_result(response)


ANSWER of genAI model: 

Hello! I'm Wonder-World, your AI travel assistant.  Here's a possible 3-day itinerary for a family trip to Paris focusing on child-friendly activities.  Remember that travel times between locations aren't included, so adjust the schedule based on your family's pace and preferred mode of transportation (metro, taxi, etc.).

**Accommodation Suggestion:**  Consider family-friendly hotels near central Paris with easy access to public transportation.  Look for hotels with amenities like cribs, connecting rooms, or kids' clubs.  Booking.com, Expedia, and other travel websites are good resources for finding options and comparing prices.


**Day 1: Fairytales and Fun**

* **Morning (9:00 AM):** Start your day at Disneyland Paris!  This is a must-do for many families visiting France.  Pre-book your tickets to avoid long queues.  Focus on Fantasyland and Adventureland, perfect for younger


#### Chain of Thoughts

Introduced in Wei et al. (2022), chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

In this exercise we want the genai model to determine if our statement is true or false.

In [21]:
# This is the statement we want to check (as false)
statement_to_evaluate = "The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1."

In [22]:
# TODO
#  Build up a chain of thoughts to help genai to create the right answer:
#
#  Use
#
#   - The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
#
#  and a corresponding explanation why this is false to help the genai model to answer
#  the following statement correctly:
#
#   - The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
chain_of_thought = (
    "The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.\n "
    "Answer: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False. \n")

chain_of_thought_prompt = (
    chain_of_thought +
    statement_to_evaluate +
    "Answer:")

In [23]:
# Call genai model for completion with chain of thought prompt as user prompt.
response = call_genai_model_for_completion(user_prompt=chain_of_thought_prompt)
print_completion_result(response)


ANSWER of genAI model: 

Adding all the odd numbers (15, 5, 13, 7, 1) gives 41.  The answer is **False**.



#### Few Shot Learning

While large-language models demonstrate remarkable zero-shot capabilities, they still fall short on more complex tasks when using the zero-shot setting. Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance.

In this exercise we want the genai model to rate a given sentence as positive or negative.
Use few shot learning to support the genai model.

In [24]:
# This is the text to rate (as negative)
text_to_rate = "What a horrible show!"

In [25]:
# TODO
#   Build up a few shot prompt that helps the genai model to determine if a given
#   statement is meant positive / negative.
few_shot_prompt = ("This is awesome! // positive \n"
          "This is bad! // negative \n"
          "Wow that movie was rad! // positive \n"
          + text_to_rate + " // \n")

In [26]:
# Call genai model for completion with few shot prompt as user prompt.
response = call_genai_model_for_completion(user_prompt=few_shot_prompt)
print_completion_result(response)


ANSWER of genAI model: 

negative



### Exercise 03: Models and parameters

In this exercise, you will learn how to use the various genai model parameters to customise the result according to your wishes.

In [27]:
# You want to know why the color of the sky is blue.
model_parameter_prompt = "Why is the sky blue?"

In [28]:
# TODO: Play around with different model parameter to get various answers
temperature = 0.5
top_k = 10
max_output_tokens = 200

In [29]:
# Call genai model for completion with few shot prompt as user prompt.
response = call_genai_model_for_completion(user_prompt=model_parameter_prompt, temperature=temperature, top_k = top_k, max_output_tokens = max_output_tokens)
print_completion_result(response)


ANSWER of genAI model: 

The sky is blue due to a phenomenon called Rayleigh scattering.  Sunlight is made up of all the colors of the rainbow.  When sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen).  These molecules are much smaller than the wavelengths of visible light.

Rayleigh scattering affects shorter wavelengths of light more strongly than longer wavelengths.  Blue and violet light have the shortest wavelengths, so they are scattered much more than other colors.  This scattered blue light is what we see when we look at the sky.

While violet light is actually scattered *more* than blue light, our eyes are more sensitive to blue, and the sun emits slightly less violet light, resulting in the sky appearing blue rather than violet.



### Exercise 04: Augmenting

In this exercise, you will learn how to use your own documents for augmenting the user prompt. To achieve this, we will first read in the files and then transfer their content to the GenAI model.

NOTE: The above-mentioned mechanism works well for content / files smaller tha 20 MB. For larger content you should upload the content to the gemini model in advance. See [genai.upload_file](https://ai.google.dev/gemini-api/docs/document-processing) for detailed information.

In [30]:
# TODO
#   Ask questions about the two short stories (see data/pdf).
#
#   Play around with the number of files, you provide and try to
#   understand the differences of the answers.

# Define location of augmenting files.
PDF_PATH_1 = PDF_DATA_PATH + "the-mystery-house-001.pdf";
PDF_PATH_2 = PDF_DATA_PATH + "the-mystery-house-002.pdf";

# Extract text from augmenting files
# This is a suitable solution for files smaller 20 MB.
pdf_file_1 = extract_text_from_pdf(PDF_PATH_1)
pdf_file_2 = extract_text_from_pdf(PDF_PATH_2)

In [31]:
# Define prompt with question about file content
augmenting_prompt = "Who is the main character of the story and what is his nickname?"

# call genai model with prompt and files
response = call_genai_model_for_completion(user_prompt=augmenting_prompt, file_list=[pdf_file_1, pdf_file_2])
print_completion_result(response)



ANSWER of genAI model: 

The main character is David Davidson, and his nickname is Shorty.

